In [26]:
import os
import pandas as pd
import numpy as np
import nltk
import praw
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from wordcloud import WordCloud, STOPWORDS 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import tree
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
import datetime as dt
from bs4 import BeautifulSoup
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report

In [2]:
# https://stackoverflow.com/questions/16206380/python-beautifulsoup-how-to-remove-all-tags-from-an-element
from bs4 import BeautifulSoup
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [3]:
flairs = ["AskIndia", "Non-Political", "[R]eddiquette", 
          "Scheduled", "Photography", "Science/Technology", "Politics",
          "Business/Finance", "Policy/Economy", "Sports", "Food", "AMA", "Coronavirus",
         "CAA-NRC-NPR" ]

In [4]:
import warnings
warnings.filterwarnings("ignore")
## Remember to add documentation before sending
from tqdm import tqdm

In [5]:
df=pd.read_csv("reddit-india-data.csv")
df.columns

Index(['flair', 'title', 'score', 'id', 'url', 'comms_num', 'created', 'body',
       'author', 'comments'],
      dtype='object')

In [6]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't", "nan","https", "www", "com" ])

In [7]:
# https://stackoverflow.com/questions/16206380/python-beautifulsoup-how-to-remove-all-tags-from-an-element
from bs4 import BeautifulSoup
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase


In [8]:
df["Text"] = df["title"].map(str) + df["body"].map(str) + df["comments"].map(str) 
preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in tqdm(df['Text'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())

100%|█████████████████████████████████████████████████████████████████████████████| 2898/2898 [00:05<00:00, 512.51it/s]


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
y=df['flair'][:]
# y = preprocessing.label_binarize(y, classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])
X=preprocessed_reviews[:]

X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.30, random_state=42)
bow = CountVectorizer()
X_train_bow=bow.fit_transform(X_train)
X_test_bow=bow.transform(X_test)
print(X_train_bow.shape, y_train.shape)

(2028, 44194) (2028,)


In [11]:
from sklearn.naive_bayes import MultinomialNB
clf=MultinomialNB()

clf.fit(X_train_bow, y_train)
y_pred = clf.predict(X_test_bow)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=flairs))

accuracy 0.44482758620689655
                    precision    recall  f1-score   support

          AskIndia       0.33      0.79      0.46        72
     Non-Political       0.14      0.48      0.21        63
     [R]eddiquette       0.53      0.63      0.58        76
         Scheduled       0.86      0.51      0.64        35
       Photography       0.86      0.10      0.18        61
Science/Technology       0.94      0.23      0.37        74
          Politics       1.00      0.09      0.17        77
  Business/Finance       0.88      0.60      0.71        62
    Policy/Economy       0.48      0.64      0.55        67
            Sports       0.40      0.45      0.42        62
              Food       0.88      0.20      0.33        74
               AMA       0.59      0.49      0.53        72
       Coronavirus       0.85      0.68      0.75        68
       CAA-NRC-NPR       0.00      0.00      0.00         7

       avg / total       0.67      0.44      0.44       870



In [12]:
from sklearn.linear_model import SGDClassifier
clf=SGDClassifier(loss='log', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)

clf.fit(X_train_bow, y_train)
y_pred = clf.predict(X_test_bow)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=flairs))

accuracy 0.41839080459770117
                    precision    recall  f1-score   support

          AskIndia       0.62      0.51      0.56        72
     Non-Political       0.13      0.87      0.22        63
     [R]eddiquette       0.54      0.64      0.59        76
         Scheduled       0.82      0.66      0.73        35
       Photography       0.73      0.31      0.44        61
Science/Technology       0.70      0.26      0.38        74
          Politics       0.96      0.31      0.47        77
  Business/Finance       0.73      0.61      0.67        62
    Policy/Economy       0.81      0.25      0.39        67
            Sports       0.53      0.29      0.38        62
              Food       0.87      0.18      0.29        74
               AMA       0.87      0.18      0.30        72
       Coronavirus       0.86      0.56      0.68        68
       CAA-NRC-NPR       0.20      0.14      0.17         7

       avg / total       0.70      0.42      0.46       870



In [13]:
clf=SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)

clf.fit(X_train_bow, y_train)
y_pred = clf.predict(X_test_bow)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=flairs))

accuracy 0.4126436781609195
                    precision    recall  f1-score   support

          AskIndia       0.57      0.54      0.55        72
     Non-Political       0.13      0.83      0.22        63
     [R]eddiquette       0.62      0.62      0.62        76
         Scheduled       0.78      0.60      0.68        35
       Photography       0.68      0.41      0.51        61
Science/Technology       0.78      0.34      0.47        74
          Politics       0.96      0.29      0.44        77
  Business/Finance       0.82      0.44      0.57        62
    Policy/Economy       0.85      0.25      0.39        67
            Sports       0.49      0.31      0.38        62
              Food       0.81      0.18      0.29        74
               AMA       0.88      0.19      0.32        72
       Coronavirus       0.79      0.56      0.66        68
       CAA-NRC-NPR       0.00      0.00      0.00         7

       avg / total       0.70      0.41      0.46       870



In [16]:
clf=tree.DecisionTreeClassifier()

clf.fit(X_train_bow, y_train)
y_pred = clf.predict(X_test_bow)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=flairs))

accuracy 0.7655172413793103
                    precision    recall  f1-score   support

          AskIndia       0.77      0.90      0.83        72
     Non-Political       0.79      0.86      0.82        63
     [R]eddiquette       0.62      0.59      0.61        76
         Scheduled       0.81      0.97      0.88        35
       Photography       0.79      0.87      0.83        61
Science/Technology       0.77      0.82      0.80        74
          Politics       0.76      0.83      0.80        77
  Business/Finance       0.91      0.85      0.88        62
    Policy/Economy       0.59      0.49      0.54        67
            Sports       0.57      0.60      0.58        62
              Food       0.84      0.76      0.79        74
               AMA       0.86      0.69      0.77        72
       Coronavirus       0.87      0.87      0.87        68
       CAA-NRC-NPR       1.00      0.29      0.44         7

       avg / total       0.77      0.77      0.76       870



In [17]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators = 1000, random_state = 42)

clf.fit(X_train_bow, y_train)
y_pred = clf.predict(X_test_bow)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=flairs))

accuracy 0.7965517241379311
                    precision    recall  f1-score   support

          AskIndia       0.72      0.94      0.81        72
     Non-Political       0.84      0.75      0.79        63
     [R]eddiquette       0.80      0.78      0.79        76
         Scheduled       0.86      0.91      0.89        35
       Photography       0.67      0.93      0.78        61
Science/Technology       0.88      0.76      0.81        74
          Politics       0.89      0.86      0.87        77
  Business/Finance       0.85      0.85      0.85        62
    Policy/Economy       0.62      0.67      0.64        67
            Sports       0.59      0.63      0.61        62
              Food       0.96      0.66      0.78        74
               AMA       0.85      0.81      0.83        72
       Coronavirus       0.98      0.88      0.93        68
       CAA-NRC-NPR       1.00      0.57      0.73         7

       avg / total       0.81      0.80      0.80       870



In [18]:
y=df['flair'][:]
# y = preprocessing.label_binarize(y, classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])
X=preprocessed_reviews[:]

X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.30, random_state=42)
tfidf = TfidfVectorizer()
X_train_tfidf=tfidf.fit_transform(X_train)
X_test_tfidf=tfidf.transform(X_test)
print(X_train_tfidf.shape, y_train.shape)

(2028, 44194) (2028,)


In [27]:
joblib.dump(tfidf, 'count_vect.pkl')

['count_vect.pkl']

In [19]:
clf=MultinomialNB()

clf.fit(X_train_tfidf, y_train)
y_pred = clf.predict(X_test_tfidf)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=flairs))

accuracy 0.5517241379310345
                    precision    recall  f1-score   support

          AskIndia       0.75      0.57      0.65        72
     Non-Political       0.27      0.62      0.38        63
     [R]eddiquette       0.47      0.75      0.58        76
         Scheduled       1.00      0.29      0.44        35
       Photography       0.85      0.57      0.69        61
Science/Technology       0.83      0.70      0.76        74
          Politics       1.00      0.12      0.21        77
  Business/Finance       0.88      0.74      0.81        62
    Policy/Economy       0.52      0.63      0.57        67
            Sports       0.26      0.79      0.40        62
              Food       0.92      0.30      0.45        74
               AMA       0.84      0.44      0.58        72
       Coronavirus       0.94      0.68      0.79        68
       CAA-NRC-NPR       0.00      0.00      0.00         7

       avg / total       0.72      0.55      0.56       870



In [20]:
clf=SGDClassifier(loss='log', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)

clf.fit(X_train_tfidf, y_train)
y_pred = clf.predict(X_test_tfidf)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=flairs))

accuracy 0.7298850574712644
                    precision    recall  f1-score   support

          AskIndia       0.77      0.83      0.80        72
     Non-Political       0.48      0.68      0.57        63
     [R]eddiquette       0.78      0.68      0.73        76
         Scheduled       0.80      0.69      0.74        35
       Photography       0.75      0.89      0.81        61
Science/Technology       0.80      0.81      0.81        74
          Politics       0.88      0.60      0.71        77
  Business/Finance       0.92      0.87      0.89        62
    Policy/Economy       0.57      0.64      0.60        67
            Sports       0.45      0.68      0.54        62
              Food       0.89      0.57      0.69        74
               AMA       0.82      0.76      0.79        72
       Coronavirus       0.94      0.87      0.90        68
       CAA-NRC-NPR       1.00      0.14      0.25         7

       avg / total       0.76      0.73      0.73       870



In [21]:
clf=SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)
clf.fit(X_train_tfidf, y_train)
y_pred = clf.predict(X_test_tfidf)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=flairs))

accuracy 0.774712643678161
                    precision    recall  f1-score   support

          AskIndia       0.77      0.89      0.83        72
     Non-Political       0.82      0.51      0.63        63
     [R]eddiquette       0.76      0.74      0.75        76
         Scheduled       0.78      0.89      0.83        35
       Photography       0.72      0.97      0.83        61
Science/Technology       0.78      0.84      0.81        74
          Politics       0.87      0.79      0.83        77
  Business/Finance       0.78      0.92      0.84        62
    Policy/Economy       0.64      0.70      0.67        67
            Sports       0.67      0.55      0.60        62
              Food       0.87      0.64      0.73        74
               AMA       0.71      0.81      0.75        72
       Coronavirus       0.94      0.91      0.93        68
       CAA-NRC-NPR       1.00      0.57      0.73         7

       avg / total       0.78      0.77      0.77       870



In [22]:
clf=tree.DecisionTreeClassifier()
clf.fit(X_train_tfidf, y_train)
y_pred = clf.predict(X_test_tfidf)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=flairs))

accuracy 0.7620689655172413
                    precision    recall  f1-score   support

          AskIndia       0.79      0.86      0.83        72
     Non-Political       0.79      0.86      0.82        63
     [R]eddiquette       0.66      0.71      0.68        76
         Scheduled       0.82      0.91      0.86        35
       Photography       0.71      0.84      0.77        61
Science/Technology       0.81      0.73      0.77        74
          Politics       0.81      0.81      0.81        77
  Business/Finance       0.90      0.90      0.90        62
    Policy/Economy       0.57      0.51      0.54        67
            Sports       0.63      0.61      0.62        62
              Food       0.80      0.72      0.76        74
               AMA       0.84      0.72      0.78        72
       Coronavirus       0.79      0.87      0.83        68
       CAA-NRC-NPR       1.00      0.29      0.44         7

       avg / total       0.76      0.76      0.76       870



In [23]:
clf=RandomForestClassifier(n_estimators = 1000, random_state = 42)
clf.fit(X_train_tfidf, y_train)
y_pred = clf.predict(X_test_tfidf)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=flairs))

accuracy 0.7977011494252874
                    precision    recall  f1-score   support

          AskIndia       0.76      0.94      0.84        72
     Non-Political       0.83      0.76      0.79        63
     [R]eddiquette       0.78      0.82      0.80        76
         Scheduled       0.84      0.89      0.86        35
       Photography       0.67      0.87      0.76        61
Science/Technology       0.87      0.78      0.82        74
          Politics       0.89      0.84      0.87        77
  Business/Finance       0.87      0.87      0.87        62
    Policy/Economy       0.67      0.66      0.66        67
            Sports       0.54      0.61      0.58        62
              Food       0.94      0.68      0.79        74
               AMA       0.83      0.82      0.83        72
       Coronavirus       0.98      0.88      0.93        68
       CAA-NRC-NPR       1.00      0.57      0.73         7

       avg / total       0.81      0.80      0.80       870



In [24]:
clf=RandomForestClassifier(n_estimators = 1500, random_state = 42)
clf.fit(X_train_tfidf, y_train)
y_pred = clf.predict(X_test_tfidf)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=flairs))

accuracy 0.7977011494252874
                    precision    recall  f1-score   support

          AskIndia       0.73      0.94      0.82        72
     Non-Political       0.82      0.75      0.78        63
     [R]eddiquette       0.80      0.79      0.79        76
         Scheduled       0.82      0.91      0.86        35
       Photography       0.68      0.87      0.76        61
Science/Technology       0.85      0.78      0.82        74
          Politics       0.90      0.84      0.87        77
  Business/Finance       0.87      0.87      0.87        62
    Policy/Economy       0.68      0.67      0.68        67
            Sports       0.54      0.61      0.57        62
              Food       0.96      0.68      0.79        74
               AMA       0.83      0.83      0.83        72
       Coronavirus       0.98      0.88      0.93        68
       CAA-NRC-NPR       1.00      0.57      0.73         7

       avg / total       0.81      0.80      0.80       870



In [40]:
clf=RandomForestClassifier(n_estimators = 500, random_state = 42)
clf.fit(X_train_tfidf, y_train)
y_pred = clf.predict(X_test_tfidf)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=flairs))

accuracy 0.7988505747126436
                    precision    recall  f1-score   support

          AskIndia       0.76      0.94      0.84        72
     Non-Political       0.80      0.76      0.78        63
     [R]eddiquette       0.78      0.79      0.78        76
         Scheduled       0.82      0.91      0.86        35
       Photography       0.68      0.89      0.77        61
Science/Technology       0.84      0.78      0.81        74
          Politics       0.89      0.83      0.86        77
  Business/Finance       0.87      0.85      0.86        62
    Policy/Economy       0.67      0.66      0.66        67
            Sports       0.60      0.63      0.61        62
              Food       0.96      0.72      0.82        74
               AMA       0.83      0.82      0.83        72
       Coronavirus       0.95      0.87      0.91        68
       CAA-NRC-NPR       1.00      0.57      0.73         7

       avg / total       0.81      0.80      0.80       870



In [41]:
# joblib.dump(clf, 'model.pkl')

['model.pkl']

In [34]:
def predict(string):
    clf = joblib.load('model.pkl')
    count_vect = joblib.load('count_vect.pkl')
    review_text = decontracted(string)
    test_vect = count_vect.transform(([review_text]))
    pred = clf.predict(test_vect)

    return pred
##########################

In [36]:
print(predict(df['Text'][0]))

['AskIndia']


In [33]:
df.head(1)

,flair,title,score,id,url,comms_num,created,body,author,comments,Text
0,AskIndia,4 days ago we had pending orders of 100 millio...,92,fwjdqr,https://www.reddit.com/r/india/comments/fwjdqr...,6,1.586290e+09,> We are getting frantic calls from our pharma...,india_ko_vanakkam,"Modi has Stockholm syndrome To be fair, the e...",4 days ago we had pending orders of 100 millio...


In [37]:
df['url'][0]

'https://www.reddit.com/r/india/comments/fwjdqr/4_days_ago_we_had_pending_orders_of_100_million/'

In [42]:
reddit = praw.Reddit(client_id='xqxZ6WzL6TC_PA', client_secret='DGlby5_lzNhqc6BkXOxfQgdxa3Y',
                     user_agent='Flair_Detector', username='Aman_Sawarn', password='izgNzhU87c$SmhT')
loaded_model = joblib.load('model.pkl')

In [45]:
def detect_flair(url):
    submission = reddit.submission(url=url)
    
    data = {}
    data['title'] = submission.title
    data['url'] = submission.url
    submission.comments.replace_more(limit=None)
    comment = ''
    for top_level_comment in submission.comments:
        comment = comment + ' ' + top_level_comment.body
    
    data["comment"] = comment
    data['title'] = decontracted(data['title'])
    data['comment'] = decontracted(data['comment'])
    data['combine'] = data['title'] + data['comment'] + data['url']
    
    return data['combine']

In [46]:
url='https://www.reddit.com/r/india/comments/fwjdqr/4_days_ago_we_had_pending_orders_of_100_million/'
detect_flair(url)

"4 days ago we had pending orders of 100 million hydroxychloroquine tablets which had been placed by state run hospitals, now suddenly we have enough of these tablets to suit the regime is agenda. Modi has Stockholm syndrome To be fair, the evidence that chloroquine works is anecdotal and not scientifically proven. The entire Story has emerged from some right wing corner in the US. A Trump supporting couple self prescribed themselves chloroquine after Trump described it as a  amiracle'. The man died and the woman was admitted to intensive care.\n\nBest to placate Trump by giving him his snake oil and keep enough for our reasonable internal use instead of stockpiling. Next thing you know he is threatening 3M India and they stop making N95 masks. It is just Trump is and mudis pr stunthttps://www.reddit.com/r/india/comments/fwjdqr/4_days_ago_we_had_pending_orders_of_100_million/"

In [50]:
type(detect_flair(url))

str

In [49]:
print(predict(detect_flair(url)))

['Coronavirus']


##### Well, till now we have been trying Random Forests which is an ensemble methood
##### Problem: The dumped pickle file has a size of 131 MB
##### It can't be pushed to github now. So, lets tune SVM as it also has an okayish performance

In [57]:
clf=SGDClassifier(loss='hinge', penalty='l1',alpha=1e-4, random_state=42, max_iter=5, tol=None)
clf.fit(X_train_tfidf, y_train)
y_pred = clf.predict(X_test_tfidf)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=flairs))

accuracy 0.7965517241379311
                    precision    recall  f1-score   support

          AskIndia       0.84      0.89      0.86        72
     Non-Political       0.80      0.75      0.77        63
     [R]eddiquette       0.75      0.75      0.75        76
         Scheduled       0.80      0.80      0.80        35
       Photography       0.81      0.97      0.88        61
Science/Technology       0.76      0.84      0.79        74
          Politics       0.80      0.82      0.81        77
  Business/Finance       0.88      0.97      0.92        62
    Policy/Economy       0.67      0.64      0.66        67
            Sports       0.63      0.60      0.61        62
              Food       0.88      0.70      0.78        74
               AMA       0.82      0.78      0.80        72
       Coronavirus       0.90      0.90      0.90        68
       CAA-NRC-NPR       1.00      0.57      0.73         7

       avg / total       0.80      0.80      0.79       870



In [58]:
joblib.dump(clf, 'model.pkl')

['model.pkl']

In [59]:
url=df['url'][100]
print(df['flair'][100])

AskIndia


In [60]:
print(predict(detect_flair(url)))

['AskIndia']


In [71]:
print(predict(X_test[3]))
print(y_test[485])

['Policy/Economy']
Scheduled


In [73]:
a=detect_flair(url)
predict(a)

array(['AskIndia'], dtype='<U18')